# init env, install packages, import libs...

In [1]:
%matplotlib inline
!pip install plotly
!pip install yahoo_fin
import pandas as pd
import numpy as np
import seaborn as sns # !
import matplotlib.pyplot as plt # !
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import plotly.express as px
from yahoo_fin.stock_info import get_data
from google.colab import drive
import requests
from datetime import datetime, date, timedelta

mergers = pd.read_csv('https://raw.githubusercontent.com/vladgrish/ds_spac_project/gh-pages/merger_spacs.csv')
pending_mergers =  pd.read_csv('https://raw.githubusercontent.com/vladgrish/ds_spac_project/gh-pages/pending_spac_mergers.csv')

# some additional settings for sns
sns.set()
sns.set(rc={"figure.figsize": (20, 10)})
PALETTE = sns.color_palette('deep', n_colors=3) # will be used for cmap which is a parameter of seaborn scatter
CMAP = ListedColormap(PALETTE.as_hex()) # A Colormap instance or registered colormap name. cmap is only used if c is an array of floats.
RANDOM_STATE = 42

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 2.4 MB/s 
     |████████████████████████████████| 83 kB 1.7 MB/s 
     |████████████████████████████████| 127 kB 45.0 MB/s 
     |████████████████████████████████| 112 kB 43.0 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=321af88a462f419f58835a693ac9d2d668e4195949f9284991f6a7436142317f
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=a8c1caf2dbb0b0aa7eff6d9580307132ee69a9077d7feed5becafdfb10a95985
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgml

# create dict of merged SPAC companies and their merge date

In [ ]:
completed_mergers = {x['Symbol']: datetime.strptime(x['SPAC merger completion date'], '%m/%d/%Y').date() for x in mergers[~mergers['SPAC merger completion date'].isna()].to_dict(orient='records')}

In [ ]:
# get all ticker changes from nasdaq

headers = {
    'authority': 'api.nasdaq.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8,ru;q=0.7',
    'origin': 'https://www.nasdaq.com',
    'referer': 'https://www.nasdaq.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36',
}

response = requests.get('https://api.nasdaq.com/api/quote/list-type-extended/symbolchangehistory', headers=headers)
symbol_changes = pd.DataFrame([(x['oldSymbol'], x['newSymbol']) for x in response.json()['data']['symbolChangeHistoryTable']['rows']], columns=['old_ticker', 'new_ticker'])

In [ ]:
# mergers that changed symbols like: {new: old,}

changed_symbols = {x['new_ticker']: x['old_ticker'] for x in symbol_changes[symbol_changes['new_ticker'].isin(mergers['Symbol'])].to_dict('records')}
len(changed_symbols.keys())

33

# read data from yahoo with yahoo finance api

In [ ]:
if 'dfs' not in globals():
  dfs = {}
merged_symbols = mergers['Symbol'].values.tolist()
pending_mergers_symbols = pending_mergers['Symbol'].values.tolist()
for x in merged_symbols+pending_mergers_symbols:
  dfs[x] = get_data(x, start_date="01/01/2008", index_as_date=True, interval='1d')

# adding additional features based on ratios within self and ratio to NASDAQ index

In [ ]:
# Adding additional metrics with explanations
min_date = date.today()
for x in dfs:
  min_date = min(dfs[x].index.min(), pd.Timestamp(min_date))
nasdaq_df = get_data('^IXIC', start_date=min_date-timedelta(days=1), index_as_date=True, interval='1d')
nasdaq_df['nasdaq_pct_change'] = nasdaq_df.close.pct_change()
for x in dfs:#[:1]:
  # add pct_change column and join it onto another df
  dfs[x] = dfs[x].join(nasdaq_df[['nasdaq_pct_change']]) # % change in NASDAQ index
  dfs[x]['low/high'] = dfs[x]['low'] / dfs[x]['high'] # low to high ratio
  dfs[x]['daily_pct_change'] = 1 - (dfs[x]['open'] / dfs[x]['close']) + 0.00001 # inter day price change (non-zero for devision)
  dfs[x]['close_pct_change'] = dfs[x].close.pct_change() # % change in closing price
  dfs[x]['pre_market_pct_change'] = dfs[x]['close_pct_change']-dfs[x]['daily_pct_change'] # difference between close % change and inter day change
  dfs[x]['nasdaq/stock_pct_change'] = dfs[x]['nasdaq_pct_change'] / dfs[x]['daily_pct_change'] # ration between NASDAQ and stock chnges

# saving all to gdrive (later will upload to github) 

In [ ]:
for x in dfs:
  dfs[x].to_csv(f'/content/drive/MyDrive/ds_project/stock_csv_data/{x}.csv')

In [ ]:
# # removing additional columns in all datasets
# for x in dfs:
#   dfs[x] = dfs[x][dfs[x].columns[:7]]

# creating some graphs

In [ ]:
#####################
### plotly graphs ###
#####################
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def get_stock_plot(x='EVEX'):
  fig = make_subplots(specs=[[{"secondary_y": True}]])
  # fig = px.line(dfs[x], x=dfs[x].index, y=['close'], title=x)
  fig.add_trace(go.Scatter(x=dfs[x].index, y=dfs[x]['close'], name="close"), secondary_y=False)
  fig.add_trace(go.Scatter(x=dfs[x].index, y=dfs[x]['volume'], name="volume"), secondary_y=True)
  fig.update_yaxes(range = [0, max(dfs[x].close.max()+5, 100)], secondary_y=False)
  if x in completed_mergers:
    fig.add_vline(x=completed_mergers['PIII'], line_dash="dash", line_color="green")
  return fig

for x in merged_symbols:
  # display(HTML(get_stock_plot(x).to_html()))
  with open(f'/content/drive/MyDrive/ds_project/merged_spac_html/{x}.html', 'w') as f:
    f.write(get_stock_plot(x).to_html(full_html=True, include_plotlyjs='cdn'))
#   # ax = sns.lineplot(data=dfs[x], x=dfs[x].index, y='close')
#   # plt.title(x)
#   # plt.show()

# README.md content formating

In [ ]:
# this was used to create the MD section of the readme for the gh-pages branch to serve in githab.io
merged_symbols.sort()
for x in merged_symbols:
  if x in completed_mergers:
    print(f'[{x} completed @ {completed_mergers[x]}](merged_spac_html/{x}.html)\n')
  else:
    print(f'[{x}](merged_spac_html/{x}.html)\n')

# getting the ready data example

In [ ]:
# reading all datasets with additional metrics from github (1 minute comparing to 8 when using yahoo api, gdrive is no better...)
# this is the final section for getting the data
def get_symbol_data(ticker):
  return pd.read_csv(f'https://raw.githubusercontent.com/vladgrish/ds_spac_project/gh-pages/stock_csv_data/{ticker}.csv', index_col=0)[1:]

if 'datasets' not in globals():
  datasets = {}

merged_symbols = mergers['Symbol'].values.tolist()
pending_mergers_symbols = pending_mergers['Symbol'].values.tolist()

for x in merged_symbols+pending_mergers_symbols:
  datasets[x] = get_symbol_data(x)